In [1]:
# to run Python-ACT-R in Notebook, we need to install the package if not done already:
try:
  from python_actr import *
except ModuleNotFoundError:
  %pip install python_actr
  from python_actr import *

# Using the code above, if python_actr is not installed, it first install python_actr 
# then imports its modules: from python_actr import *
# so no needs to import the following modules one by one, but you can use them instead 
# by uncommenting one by one, when needed, instead of importing everything at once:
#import python_actr
#from python_actr import log
#from python_actr import ACTR
#from python_actr import Model
#from python_actr import Buffer
#from python_actr import Memory
#from python_actr import DMSpreading
#from python_actr import log_everything

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for python_actr: filename=python_actr-1.9.2-py3-none-any.whl size=30925 sha256=7875d88ad2699f2a8fd3b8daa90b84e2b24ce9ff50e572ecbd97a15f3fc200b2
  Stored in directory: /root/.cache/pip/wheels/09/90/92/c6869eae4a55eaa262b06436143c2bbfa9ab4b466d84ad0a8d
Successfully built python_actr
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
class MyEnvironment(Model):
    pass

#####
# create an act-r agent

class MyAgent(ACTR):
    
    focus=Buffer()

    production_time=0.02         # production parameter settings
    production_sd=0.10
    production_threshold=-20

    pm_new=PMNew(alpha=0.2)      # The new TD-inspired utility learning system

#### NOTE - this model uses an alternative way of writing chunks using a list instead of a dictionary form
#### so the order is what gives it meaning. This is just an alternative way of writing them, they work just the same

    def init():
        focus.set('state:engine_off goal:run_engine')

    def openDoor(focus='state:engine_off goal:run_engine'):  
      print (" ")   
      print ("Let start this engine!")          
      focus.set('state:engine_off action:start_car')             

    def sit(focus='state:engine_off action:start_car'):   
      print (" ")       
      print ("I'm going to start this car....")    
      focus.set('state:choose goal:run_engine action:unknown')
                            ## following this the agent will choose to explore or not

## choose to explore different actions

    def explore(focus='state:choose goal:run_engine action:unknown'):
      print (" ")       
      print ("Let's explore...")   
      focus.set('state:explore goal:run_engine action:select')

    def exploreKey(focus='state:explore goal:run_engine action:select'):  
      print (" ")        
      print ("I'll turn the key")
      focus.set('state:make goal:run_engine action:turn_key')
      self.reward(0.0)    										# reward reset, no inherited reward

    def exploreReevaluate(focus='state:explore goal:run_engine action:select'):  
      print (" ")     
      print ("Going to reevaluate")
      focus.set('state:make goal:run_engine action:reevaluate')
      self.reward(0.0)  											# reward reset, no inherited rewardQ

## choose not to explore different ways

    def donotexplore(focus='state:choose ingrediant:ham varient:unknown'):
      print (" ")          
      print ("I am feeling rational in a narrow sort of way")    
      focus.set('state:donotexplore ingrediant:ham varient:select')
      self.reward(0.0)  # reward reset, no inherited reward

    def chooseKey(focus='state:make goal:run_engine action:turn_key'): # will inherit rewarded from ham
      print (" ")         
      print ("I will turn the key...")    
      focus.set('state:make goal:run_engine action:turnED_key')

    def chooseReevaluate(focus='state:make goal:run_engine action:reevaluate'): # will inherit rewared from parma_ham 
      print (" ")        
      print ("I will reevaluate...")    
      focus.set('state:make goal:run_engine action:reevaluatedED')

## resume car starting activity

    def Key(focus='state:make goal:run_engine action:turnED_key'):
      print (" ")
      print ("I have turnED the key")
      focus.set('state:engine_off goal:run_engine')
      self.reward(0.3)                                              

    def Reevaluate(focus='state:make goal:run_engine action:reevaluatED'):
      print (" ")
      print ("I have now reevaluatED ************")
      focus.set('state:engine_off goal:run_engine')
      self.reward(0.5)                                                 

    def CarStart(focus='state:on goal:in_progress'):
      print (" ")
      print ("I am making progress to starting this car")
      focus.set('state:engine_off goal:run_engine')
      self.reward(0.0)   # reset

                   

tim=MyAgent()                              
subway=MyEnvironment()                    
subway.agent=tim                          
log_everything(subway)                 
subway.run(3)                             

   0.000 agent.production_match_delay 0
   0.000 agent.production_sd 0.1
   0.000 agent.production_threshold -20
   0.000 agent.production_time 0.02
   0.000 agent.production_time_sd None
   0.000 agent.focus.chunk None
   0.000 agent.pm_new.alpha 0.2
   0.000 agent.pm_new.clearFlag False
   0.000 agent.focus.chunk goal:run_engine state:engine_off
   0.000 agent.production openDoor
   0.020 agent.production None
 
Let start this engine!
   0.020 agent.focus.chunk action:start_car state:engine_off
   0.020 agent.production sit
   0.040 agent.production None
 
I'm going to start this car....
   0.040 agent.focus.chunk action:unknown goal:run_engine state:choose
   0.040 agent.production explore
   0.060 agent.production None
 
Let's explore...
   0.060 agent.focus.chunk action:select goal:run_engine state:explore
   0.060 agent.production exploreKey
   0.080 agent.production None
 
I'll turn the key
   0.080 agent.focus.chunk action:turn_key goal:run_engine state:make
   0.080 agent.pm_n

In [ ]:
#################### ham cheese production model ###################

# this is the simplest type of act-r model
# it uses only the production system and one buffer
# the buffer represents the focus of thought
# we call it the focus buffer but it is often called the goal buffer
# productions fire if they match the focus buffer
# each production changes the contents of focus buffer so a different production will fire on the next cycle


#####
# Python ACT-R requires an environment
# but in this case we will not be using anything in the environment
# so we 'pass' on putting things in there

class MyEnvironment(Model):
    pass

#####
# create an act-r agent

class MyAgent(ACTR):
    DMbuffer=Buffer()                    
    DM=Memory(DMbuffer)                   
    focus=Buffer()

    production_time=0.03         # production parameter settings
    production_sd=0.01
    production_threshold=-10

    pm_new=PMNew(alpha=0.2)      # The new TD-inspired utility learning system

## this is based on the ACT-R model "parma-ham"
    def init():
      DM.add('action1:turn_key')                  
      DM.add('action2:reevaluate')
      focus.set('buffer 1a')

    def startCar(focus='buffer 1a'):
      print ("Trying to start car - WHAT SHOULD I DO?")
      focus.set('buffer 1')    # two matching conditions: functions "decision_1" & "decision_2"

    def decision_1a(focus='buffer 1'):   
      print (" ")
      print ("Requesting action 1...")
      DM.request('action1:?')             
      focus.set('buffer 2')             

    def decision_1b(focus='buffer 2', DMbuffer='action1:?action1'): 
      print (" ")     
      print (action1)        
      focus.set('buffer 1') 	# LOOP to buffer 1


    def decision_2a(focus='buffer 1'):
      print (" ")
      print ("Requesting action 2...")
      DM.request('action2:?')             
      focus.set('buffer 3')
      self.reward(0.1)

    def decision_2b(focus='buffer 3', DMbuffer='action2:?action2'):
      print (" ")
      print (action2)          
      focus.set('buffer 1') 	# LOOP to buffer 1

    def do_nothing(focus='buffer done'):
      print ("I'm done")	
      self.stop()

tim=MyAgent()                              
subway=MyEnvironment()                    
subway.agent=tim                          
log_everything(subway)                 
subway.run(20)     # run for 5 seconds to learn that param ham is more rewarding to use     

   3.000 agent.production_match_delay 0
   3.000 agent.production_sd 0.01
   3.000 agent.production_threshold -10
   3.000 agent.production_time 0.03
   3.000 agent.production_time_sd None
   3.000 agent.DM.error False
   3.000 agent.DM.busy False
   3.000 agent.DM.latency 0.05
   3.000 agent.DM.threshold 0
   3.000 agent.DM.maximum_time 10.0
   3.000 agent.DM.record_all_chunks False
   3.000 agent.DMbuffer.chunk None
   3.000 agent.focus.chunk None
   3.000 agent.pm_new.alpha 0.2
   3.000 agent.pm_new.clearFlag False
   3.000 agent.focus.chunk buffer 1a
   3.000 agent.production startCar
   0.030 agent.production None
Trying to start car - WHAT SHOULD I DO?
   0.030 agent.focus.chunk buffer 1
   0.030 agent.production decision_1a
   0.060 agent.production None
 
Requesting action 1...
   0.060 agent.DM.busy True
   0.060 agent.focus.chunk buffer 2
   0.110 agent.DMbuffer.chunk action1:turn_key
   0.110 agent.DM.busy False
   0.110 agent.production decision_1b
   0.140 agent.production

In [ ]:
class MyEnv(Model):
  pass
class MyAgent(ACTR):
  production_time = 0.05
  production_sd = 0.01
  production_threshold = -20
  
  goal = Buffer() # Creating the goal buffer for the agent
  
  def init(): # this rule fires when the agent is instantiated.
    goal.set("sandwich bread") # set goal buffer to direct program flow
  def bread_bottom(goal="sandwich bread"): # if goal="sandwich bread" , fire rule
    print ("I have a piece of bread")
    goal.set("stop") # set goal buffer to direct program flow
  def stop_production(goal="stop"):
    self.stop() # stop the agent

tim = MyAgent()
subway=MyEnv()
subway.agent=tim
log_everything(subway)
subway.run()

   3.000 agent.production_match_delay 0
   3.000 agent.production_sd 0.01
   3.000 agent.production_threshold -20
   3.000 agent.production_time 0.05
   3.000 agent.production_time_sd None
   3.000 agent.goal.chunk None
   3.000 agent.goal.chunk sandwich bread
   3.000 agent.production bread_bottom
   0.050 agent.production None
I have a piece of bread
   0.050 agent.goal.chunk stop
   0.050 agent.production stop_production
   0.100 agent.production None


# New section

In [ ]:
class MyEnv(Model):
  pass
class MyAgent(ACTR):
  goal=Buffer()
  retrieve=Buffer()
  memory=Memory(retrieve)
  
  def init():
    memory.add('count 0 1')
    memory.add('count 1 2')
    memory.add('count 2 3')
    memory.add('count 3 4')
    memory.add('count 4 5')
    memory.add('count 5 6')
    memory.add('count 6 7')
    memory.add('count 7 8')
    memory.add('count 8 9')
    memory.add('count 9 10')
    goal.set('countFrom 2 5 starting')
  def start(goal='countFrom ?start ?end starting'):
    memory.request('count ?start ?next')
    goal.set('countFrom ?start ?end counting')

  def increment(goal='countFrom ?x !?x counting',
                retrieve='count ?x ?next'):
    print (x)
    memory.request('count ?next ?nextNext')
    goal.modify(_1=next)

  def stop(goal='countFrom ?x ?x counting'):
    print (x)
    goal.set('countFrom ?x ?x stop')

#model=Count()  #MyAgent()
#model.goal.set('countFrom 2 5 starting')
#model.run()

tim = MyAgent() 
subway=MyEnv()
subway.agent=tim
log_everything(subway)
subway.run()

   0.100 agent.production_match_delay 0
   0.100 agent.production_threshold None
   0.100 agent.production_time 0.05
   0.100 agent.production_time_sd None
   0.100 agent.goal.chunk None
   0.100 agent.memory.error False
   0.100 agent.memory.busy False
   0.100 agent.memory.latency 0.05
   0.100 agent.memory.threshold 0
   0.100 agent.memory.maximum_time 10.0
   0.100 agent.memory.record_all_chunks False
   0.100 agent.retrieve.chunk None
   0.100 agent.goal.chunk countFrom 2 5 starting
   0.100 agent.production start
   0.050 agent.production None
   0.050 agent.memory.busy True
   0.050 agent.goal.chunk countFrom 2 5 counting
   0.100 agent.retrieve.chunk count 2 3
   0.100 agent.memory.busy False
   0.100 agent.production increment
   0.150 agent.production None
2
   0.150 agent.memory.busy True
   0.150 agent.goal.chunk countFrom 3 5 counting
   0.200 agent.retrieve.chunk count 3 4
   0.200 agent.memory.busy False
   0.200 agent.production increment
   0.250 agent.production None


In [ ]:
class MyEnv(Model):
  pass
class MyAgent(ACTR):
  goal=Buffer()
  retrieve=Buffer()
  memory=Memory(retrieve)
  
  def init():
    memory.add('count 0 1')
    memory.add('count 1 2')
    memory.add('count 2 3')
    memory.add('count 3 4')
    memory.add('count 4 5')
    memory.add('count 5 6')
    memory.add('count 6 7')
    memory.add('count 7 8')
    goal.set('add 5 2 count:None sum:None')
  def initializeAddition(goal='add ?num1 ?num2 count:None?count sum:None?sum'):
    goal.modify(count=0,sum=num1)
    memory.request('count ?num1 ?next')
    
  def terminateAddition(goal='add ?num1 ?num2 count:?num2 sum:?sum'):
    goal.set('result ?sum')
    #print (sum)
    
  def incrementSum(goal='add ?num1 ?num2 count:?count!?num2 sum:?sum',
                   retrieve='count ?sum ?next'):
    goal.modify(sum=next)
    memory.request('count ?count ?n2')
    
  def incrementCount(goal='add ?num1 ?num2 count:?count sum:?sum',
                     retrieve='count ?count ?next'):
    goal.modify(count=next)
    memory.request('count ?sum ?n2')

tim = MyAgent() 
subway=MyEnv()
subway.agent=tim
log_everything(subway)
subway.run()

   3.350 agent.production_match_delay 0
   3.350 agent.production_threshold None
   3.350 agent.production_time 0.05
   3.350 agent.production_time_sd None
   3.350 agent.goal.chunk None
   3.350 agent.memory.error False
   3.350 agent.memory.busy False
   3.350 agent.memory.latency 0.05
   3.350 agent.memory.threshold 0
   3.350 agent.memory.maximum_time 10.0
   3.350 agent.memory.record_all_chunks False
   3.350 agent.retrieve.chunk None
   3.350 agent.goal.chunk add 5 2 count:None sum:None
   3.350 agent.production initializeAddition
   0.050 agent.production None
   0.050 agent.goal.chunk add 5 2 count:0 sum:None
   0.050 agent.goal.chunk add 5 2 count:0 sum:5
   0.050 agent.memory.busy True
   0.100 agent.retrieve.chunk count 5 6
   0.100 agent.memory.busy False
   0.100 agent.production incrementSum
   0.150 agent.production None
   0.150 agent.goal.chunk add 5 2 count:0 sum:6
   0.150 agent.memory.busy True
   0.200 agent.retrieve.chunk count 0 1
   0.200 agent.memory.busy False

In [ ]:
class MyEnv(Model):
  pass
class MyAgent(ACTR):
  goal=Buffer()
  retrieve=Buffer()
  memory=Memory(retrieve)  
  
  def init():
    memory.add('addfact 3 4 7')
    memory.add('addfact 6 7 13')
    memory.add('addfact 10 3 13')
    memory.add('addfact 1 7 8')
    goal.set('add 3 6 4 7 None None None')
  def startPair(goal='add ? ?one1 ? ?one2 ? None?ans ?'):
    goal.modify(_6='busy')
    memory.request('addfact ?one1 ?one2 ?')
    
  def addOnes(goal='add ? ? ? ? ? busy?ans ?carry', retrieve='addfact ? ? ?sum'):
    goal.modify(_6=sum,_7='busy')
    memory.request('addfact 10 ? ?sum')
    
  def processCarry(goal='add ?ten1 ? ?ten2 ? None?tenAns ?oneAns busy?carry',retrieve='addfact 10 ?rem ?sum'):
    goal.modify(_6=rem,_7=1,_5='busy')
    memory.request('addfact ?ten1 ?ten2 ?')
    
  def noCarry(goal='add ?ten1 ? ?ten2 None?tenAns ?oneAns busy?carry',memory='error:True'):
    goal.modify(_6=0,_4='busy')
    memory.request('addfact ?ten1 ?ten2 ?')
    
  def addTensDone(goal='add ? ? ? ? busy?tenAns ?oneAns 0',retrieve='addfact ? ? ?sum'):
    #print (sum,oneAns)
    goal.modify(_5=sum)
  def addTensCarry(goal='add ? ? ? ? busy?tenAns ? 1?carry',retrieve='addfact ? ? ?sum'):
    goal.modify(_7=0)
    memory.request('addfact 1 ?sum ?')

  
tim = MyAgent() 
subway=MyEnv()
subway.agent=tim
log_everything(subway)
subway.run()

   3.450 agent.production_match_delay 0
   3.450 agent.production_threshold None
   3.450 agent.production_time 0.05
   3.450 agent.production_time_sd None
   3.450 agent.goal.chunk None
   3.450 agent.memory.error False
   3.450 agent.memory.busy False
   3.450 agent.memory.latency 0.05
   3.450 agent.memory.threshold 0
   3.450 agent.memory.maximum_time 10.0
   3.450 agent.memory.record_all_chunks False
   3.450 agent.retrieve.chunk None
   3.450 agent.goal.chunk add 3 6 4 7 None None None
   3.450 agent.production startPair
   0.050 agent.production None
   0.050 agent.goal.chunk add 3 6 4 7 None busy None
   0.050 agent.memory.busy True
   0.100 agent.retrieve.chunk addfact 6 7 13
   0.100 agent.memory.busy False
   0.100 agent.production addOnes
   0.150 agent.production None
   0.150 agent.goal.chunk add 3 6 4 7 None 13 None
   0.150 agent.goal.chunk add 3 6 4 7 None 13 busy
   0.150 agent.memory.busy True
   0.200 agent.retrieve.chunk addfact 10 3 13
   0.200 agent.memory.busy F

In [ ]:
class MyEnv(Model):
  pass
class MyAgent(ACTR):
  goal=Buffer()
  retrieve=Buffer()
  memory=Memory(retrieve)

  text=TextOutput()
  
  def init():
    memory.add('property shark dangerous true')
    memory.add('property shark locomotion swimming')
    memory.add('property shark category fish')
    memory.add('property fish category animal')
    memory.add('property bird category animal')
    memory.add('property canary category bird')
    goal.set('isMember shark fish result:None')
  def initialRetrieve(goal='isMember ?obj ?cat result:None'):
    goal.modify(result='pending')
    memory.request('property ?obj category ?')

  def directVerify(goal='isMember ?obj ?cat result:pending',
                   retrieve='property ?obj category ?cat'):
    goal.modify(result='yes')
    text.write('Yes')

  def chainCategory(goal='isMember ?obj1 ?cat result:pending',
                    retrieve='property ?obj1 category ?obj2!?cat'):
    goal.modify(_1=obj2)
    memory.request('property ?obj2 category ?')

  def fail(goal='isMember ?obj1 ?cat result:pending',memory='error:True'):
    goal.modify(result='no')
    text.write('No')
    
tim = MyAgent() 
subway=MyEnv()
subway.agent=tim
log_everything(subway)
subway.run()

   3.300 agent.production_match_delay 0
   3.300 agent.production_threshold None
   3.300 agent.production_time 0.05
   3.300 agent.production_time_sd None
   3.300 agent.goal.chunk None
   3.300 agent.memory.error False
   3.300 agent.memory.busy False
   3.300 agent.memory.latency 0.05
   3.300 agent.memory.threshold 0
   3.300 agent.memory.maximum_time 10.0
   3.300 agent.memory.record_all_chunks False
   3.300 agent.retrieve.chunk None
   3.300 agent.goal.chunk isMember shark fish result:None
   3.300 agent.production initialRetrieve
   0.050 agent.production None
   0.050 agent.goal.chunk isMember shark fish result:pending
   0.050 agent.memory.busy True
   0.100 agent.retrieve.chunk property shark category fish
   0.100 agent.memory.busy False
   0.100 agent.production directVerify
   0.150 agent.production None
   0.150 agent.goal.chunk isMember shark fish result:yes
Yes
   0.150 agent.text Yes


In [ ]:
class MyEnv(Model):
    def start(self):
        self.count=0
        yield 1   # wait one second
        self.letter=Model(isa='letter',x=0.5,y=0.5,visible=True)
        self.letter.text=self.random.choice('BCDFGHJKLMNPQRSTVWXYZ')
        self.target=self.letter.text

    def press(self,key):
        self.pressed=key
        if key==self.target:
            log.success=True
        else:
            log.success=False
        self.letter.visible=False
        self.count+=1
        if self.count==10:
            self.stop()
        else:
            yield 1
            self.letter.text=self.random.choice('BCDFGHJKLMNPQRSTVWXYZ')
            self.letter.visible=True
            self.target=self.letter.text
        
# Define the ACT-R Model    
class MyAgent(ACTR):
  goal=Buffer()
  visual=Buffer()
  vision=SOSVision(visual)
  goal.set('attend')
  def findUnattendedLetter(goal='attend',vision='busy:False'):
    vision.request('isa:letter')
    goal.set('attend')
    
  def encodeLetter(goal='attend',visual='isa:letter text:?letter'):
    goal.set('respond ?letter')
    
  def respond(goal='respond ?letter'):
    self.parent.press(letter)
    visual.clear()
    goal.set('attend')

tim=MyAgent
subway=MyEnv()

subway.agent=MyAgent()
log_everything(subway)
subway.run()

NameError: ignored

In [ ]:
pip install python_actr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for python-actr: filename=python_actr-1.9.2-py3-none-any.whl size=30928 sha256=0903eb82e05df9b5d296a08b9ec5260d8cbb91e106bb9d1eb2edf361ce310aee
  Stored in directory: /root/.cache/pip/wheels/8c/74/5a/39db992e02dc2199445e11d5dceb3df9c573130380bb553ae4
Successfully built python-actr


In [ ]:
#https://github.com/rlwest/Learning-Models/blob/master/simple_environment_v1.py
##simple env
#################### ham cheese production model ###################


##import ccm      
##log=ccm.log(html=True)   

##from ccm.lib.actr import *  

from python_actr import *  

class Subway(Model):        # items in the environment look and act like chunks - but note the syntactic differences
    bread=Model(isa='bread',location='on_counter')
    cheese=Model(isa='cheese',location='on_counter')
    ham=Model(isa='ham',location='on_counter')
    bread_top=Model(isa='bread_top',location='on_counter')

class MotorModule(Model):     # create a motor module do the acti Ml ons 
    def do_bread(self):           # note that technically the motor module is outside the agent
        yield 2                   # but it is controlled from within the agent, i.e., it bridges the cognitive and the environment
        print ("do the bread")
        self.parent.parent.bread.location='on_plate'    # self=MotorModule, parent=MyAgent, parent of parent=Subway
    def do_cheese(self):     
        yield 2                   # yield refers to how long the action takes, but cognition can continue while waiting for an action to complete
        print ("do the cheese")
        self.parent.parent.cheese.location='on_plate'   # in this case the motor actions make changes to the environment objects
    def do_ham(self):     
        yield 2
        print ("do the ham")
        self.parent.parent.ham.location='on_plate'
    def do_bread_top(self):     
        yield 2
        print ("do the bread on top")
        self.parent.parent.bread_top.location='on_plate'
        
class MyAgent(ACTR):    
    focus=Buffer()
    motor=MotorModule()

    def init():
        focus.set('sandwich bread')

    def bread_bottom(focus='sandwich bread'):
        print ("I have a piece of bread")     
        focus.set('sandwich cheese')
        motor.do_bread()                  # direct the motor module to do an action

    def cheese(focus='sandwich cheese', bread='location:on_plate'):   # production fires off the environment directly
        print ("I have cheese")                                         # this is legitimate if it is assumed that the agent is... 
        focus.set('sandwich ham')                                     # continuously and successfully monitoring the envionment
        motor.do_cheese()                                             # and time for monitoring is incorporated into the action time

    def ham(focus='sandwich ham', cheese='location:on_plate'):        # slot name required for objects
        print ("I have ham")
        focus.set('sandwich bread_top')
        motor.do_ham()

    def bread_top(focus='sandwich bread_top', ham='location:on_plate'):
        print ("I have bread")
        focus.set('stop')
        motor.do_bread_top()

    def stop_production(focus='stop', bread_top='location:on_plate'):  # wait for the action to complete before stopping
        print ("I have made a ham and cheese sandwich")
        self.reward(1)
        self.stop()


tim=MyAgent()
env=Subway()
env.agent=tim 
log_everything(env)
print(tim.reward)
env.run()
##ccm.finished()

   8.250 agent.production_match_delay 0
   8.250 agent.production_threshold None
   8.250 agent.production_time 0.05
   8.250 agent.production_time_sd None
   8.250 agent.focus.chunk None
   8.250 bread.isa bread
   8.250 bread.location on_counter
   8.250 bread_top.isa bread_top
   8.250 bread_top.location on_counter
   8.250 cheese.isa cheese
   8.250 cheese.location on_counter
   8.250 ham.isa ham
   8.250 ham.location on_counter
   8.250 agent.focus.chunk sandwich bread
   8.250 agent.production bread_bottom
   0.050 agent.production None
I have a piece of bread
   0.050 agent.focus.chunk sandwich cheese
do the bread
   2.050 bread.location on_plate
   2.050 agent.production cheese
   2.100 agent.production None
I have cheese
   2.100 agent.focus.chunk sandwich ham
do the cheese
   4.100 cheese.location on_plate
   4.100 agent.production ham
   4.150 agent.production None
I have ham
   4.150 agent.focus.chunk sandwich bread_top
do the ham
   6.150 ham.location on_plate
   6.150 age

In [ ]:
size=1
trials=10
display_time=5
#PairedExperiment
class MyEnv(Model): 
  word=Model(visible=False,x=0.5,y=0.5,font='Arial 20',type='Text')
  number=Model(visible=False,x=0.5,y=0.5,font='Arial 20',type='Number')
  
  def start(self):
      pairs=[('bank','0'),('card','1'),('dart','2'),('face','3'),
             ('game','4'),('hand','5'),('jack','6'),('king','7'),
             ('lamb','8'),('mask','9'),('neck','0'),('pipe','1'),
             ('quip','2'),('rope','3'),('sock','4'),('tent','5'),
             ('vent','6'),('wall','7'),('xray','8'),('zinc','9')]
      items=self.random.sample(pairs,size)      
      scores=[]
      times=[]
      for i in range(trials):
        score=0
        time=0
        self.random.shuffle(items)
        for w,n in items:
          self.word.text=w
          self.word.visible=True
          start=self.now()
          self.key=None
          yield self.key_pressed,display_time
          self.word.visible=False
          
          if self.key==n:
            score+=1
            time+=self.now()-start
          self.number.text=n
          self.number.visible=True
          yield display_time
          self.number.visible=False
        scores.append(score)
        times.append(time)
      log.scores=scores
      log.times=times      
   
  def key_pressed(self,key):
    self.key=key      
        
        
      

    
      
      
class Paired(ACTR):
  goal=Buffer()
  retrieve=Buffer()
  memory=Memory(retrieve,threshold=-2,latency=0.35)
  DMBaseLevel(memory)
  DMNoise(memory,noise=0.5)
  
  visual=Buffer()
  location=Buffer()
  vision=Vision(visual,location)    
  
  motor=Motor()
  
  def attendProbe(goal='state:start',vision='busy:False',location='?x ?y'):
    vision.examine('?x ?y')
    goal.modify(state='attendingProbe')
    location.clear()
    
  def detectStudyItem(goal='state:readStudyItem word:?w!None',vision='busy:False',location='?x ?y'):
    vision.examine('?x ?y')
    goal.modify(state='attendingTarget')
    location.clear()
    
  def associate(goal='state:attendingTarget word:?word!None num:?num',visual='type:Number text:?text'):
    memory.add('word:?word num:?text')
    visual.clear()
    goal.set('state:start word:None num:None')
    
  def readProbe(goal='state:attendingProbe word:None',visual='type:Text text:?word'):
    memory.request('word:?word')
    goal.modify(state='testing',word=word)
    visual.clear()
    
  def recall(goal='state:testing word:?word',retrieve='word:?word num:?num',motor='busy:False'):
    motor.press(num)
    goal.modify(state='readStudyItem')
    
  def cannotRecall(goal='state:testing word:?word',memory='error:True'):
    goal.modify(state='readStudyItem')

In [ ]:
print(len('100'))

3


# New section 2

In [ ]:
#https://github.com/rlwest/Learning-Models/blob/master/simple_agents_v1.py
#################### ham cheese production model ###################


##import ccm      
##log=ccm.log(html=True)   

##from ccm.lib.actr import *  
from python_actr import *  

class Subway(Model):        # items in the environment look and act like chunks - but note the syntactic differences
    bread=Model(isa='bread',location='on_counter')
    cheese=Model(isa='cheese',location='on_counter')
    ham=Model(isa='ham',location='on_counter')
    bread_top=Model(isa='bread_top',location='on_counter')

    sue_voice=Model(isa='voice',message='none')      # sounds are part of the environment

class MotorModule(Model):     # create a motor module do the actions 
    def do_bread(self):           # note that technically the motor module is outside the agent
        yield 2                   # but it is controlled from within the agent, i.e., it bridges the cognitive and the environment
        print ("do the bread")
        self.parent.parent.bread.location='on_plate'    # self=MotorModule, parent=MyAgent, parent of parent=Subway
    def do_cheese(self):     
        yield 2                   # yield refers to how long the action takes, but cognition can continue while waiting for an action to complete
        print ("do the cheese")
        self.parent.parent.cheese.location='on_plate'   # in this case the motor actions make changes to the environment objects
    def do_ham(self):     
        yield 2
        print ("do the ham")
        self.parent.parent.ham.location='on_plate'
    def do_bread_top(self):     
        yield 2
        print ("do the bread on top")
        self.parent.parent.bread_top.location='on_plate'

    def sue_speaks(self):       # create a sound object
        yield 2
        print ("sue is speaking")
        self.parent.parent.sue_voice.message='finished'
        
class MyAgent(ACTR):    
    focus=Buffer()
    motor=MotorModule()

    def init():
        focus.set('sandwich bread')

    def bread_bottom(focus='sandwich bread'):
        print ("tim - I have a piece of bread")     
        focus.set('sandwich cheese')
        motor.do_bread()                  # direct the motor module to do an action

    def cheese(focus='sandwich cheese', bread='location:on_plate'):   # production fires off the environment directly
        print ("tim - I have cheese")                                         # this is legitimate if it is assumed that the agent is... 
        focus.set('sandwich ham')                                     # continuously and successfully monitoring the envionment
        motor.do_cheese()                                             # and time for monitoring is incorporated into the action time

    def stop_production(sue_voice='message:finished'):                # wait for sue to say she is finished
        print ("tim - we are done")
        self.stop()                        # stops the whole thing, not just the agent

class MyAgent2(ACTR):    
    focus=Buffer()
    motor=MotorModule()

    def init():
        focus.set('sandwich ham')

    def ham(focus='sandwich ham', cheese='location:on_plate'):        # slot name required for objects
        print ("sue - I have ham")
        focus.set('sandwich bread_top')
        motor.do_ham()

    def bread_top(focus='sandwich bread_top', ham='location:on_plate'):
        print ("sue - I have bread")
        focus.set('finished')
        motor.do_bread_top()

    def finished(focus='finished', bread_top='location:on_plate'):  # wait for the action to complete before stopping
        print ("sue - we have made a ham and cheese sandwich")
        print ("sue - finished")
        focus.set('stop')
        motor.sue_speaks()                   # says she is finished

tim=MyAgent()
sue=MyAgent2()
env=Subway()
env.agent=tim
env.agent=sue

log_everything(env)

env.run()
#ccm.finished()

   8.250 agent.production_match_delay 0
   8.250 agent.production_threshold None
   8.250 agent.production_time 0.05
   8.250 agent.production_time_sd None
   8.250 agent.focus.chunk None
   8.250 bread.isa bread
   8.250 bread.location on_counter
   8.250 bread_top.isa bread_top
   8.250 bread_top.location on_counter
   8.250 cheese.isa cheese
   8.250 cheese.location on_counter
   8.250 ham.isa ham
   8.250 ham.location on_counter
   8.250 sue_voice.isa voice
   8.250 sue_voice.message none
   8.250 agent.focus.chunk sandwich ham
tim - I have a piece of bread
do the bread
   2.050 bread.location on_plate
tim - I have cheese
do the cheese
   4.100 cheese.location on_plate
   4.100 agent.production ham
   4.150 agent.production None
sue - I have ham
   4.150 agent.focus.chunk sandwich bread_top
do the ham
   6.150 ham.location on_plate
   6.150 agent.production bread_top
   6.200 agent.production None
sue - I have bread
   6.200 agent.focus.chunk finished
do the bread on top
   8.200 b

In [ ]:
#https://github.com/rlwest/Learning-Models/blob/master/simple_DM_v1.py
#################### ham cheese production DM model ###################

# this builds on the production model
# two productions are added
# the first requests that the declarative memory module retrieves the condiment that the cutomer ordered
# which is stored in declarative memory
# the second production fires when this has happened


##import ccm      
##log=ccm.log()   

##from ccm.lib.actr import *  
from python_actr import *  

#####
# Python ACT-R requires an environment
# but in this case we will not be using anything in the environment
# so we 'pass' on putting things in there

class MyEnvironment(Model):
    pass

#####
# create an act-r agent


class MyAgent(ACTR):
    focus=Buffer()
    DMbuffer=Buffer()                           # create a buffer for the declarative memory (henceforth DM)
    DM=Memory(DMbuffer)                         # create DM and connect it to its buffer    
    
    def init():
        DM.add('condiment:mustard condiment1:mustard1 condiment2:mustard2')              # put a chunk into DM
        focus.set('sandwich bread')
        
    def bread_bottom(focus='sandwich bread'):   
        print ("I have a piece of bread")         
        focus.set('sandwich cheese')    

    def cheese(focus='sandwich cheese'):        
        print ("I have put cheese on the bread")  
        focus.set('sandwich ham')

    def ham(focus='sandwich ham'):
        print ("I have put  ham on the cheese")
        focus.set('get_condiment')

    def condiment(focus='get_condiment'):
        print ("recalling the order")
        DM.request('condiment1:?')                # retrieve a chunk from DM into the DM buffer
        focus.set('sandwich condiment')         # ? means that slot can match any content

    def order(focus='sandwich condiment', DMbuffer='condiment1:?condiment1'):  # match to DMbuffer as well
        print ("I recall they wanted......." )                                # put slot 2 value in ?condiment
        print (condiment1)             
        print ("i have put the condiment on the sandwich")
        focus.set('sandwich bread_top')

    def bread_top(focus='sandwich bread_top'):
        print ("I have put bread on the ham")
        print ("I have made a ham and cheese sandwich")
        focus.set('stop')   

    def stop_production(focus='stop'):
        self.stop()


tim=MyAgent()                              # name the agent
subway=MyEnvironment()                     # name the environment
subway.agent=tim                           # put the agent in the environment
log_everything(subway)                 # print out what happens in the environment

subway.run()                               # run the environment
#ccm.finished()                             # stop the environment

  10.300 agent.production_match_delay 0
  10.300 agent.production_threshold None
  10.300 agent.production_time 0.05
  10.300 agent.production_time_sd None
  10.300 agent.DM.error False
  10.300 agent.DM.busy False
  10.300 agent.DM.latency 0.05
  10.300 agent.DM.threshold 0
  10.300 agent.DM.maximum_time 10.0
  10.300 agent.DM.record_all_chunks False
  10.300 agent.DMbuffer.chunk None
  10.300 agent.focus.chunk None
  10.300 agent.focus.chunk sandwich bread
  10.300 agent.production bread_bottom
   0.050 agent.production None
I have a piece of bread
   0.050 agent.focus.chunk sandwich cheese
   0.050 agent.production cheese
   0.100 agent.production None
I have put cheese on the bread
   0.100 agent.focus.chunk sandwich ham
   0.100 agent.production ham
   0.150 agent.production None
I have put  ham on the cheese
   0.150 agent.focus.chunk get_condiment
   0.150 agent.production condiment
   0.200 agent.production None
recalling the order
   0.200 agent.DM.busy True
   0.200 agent.foc

In [ ]:
#https://github.com/rlwest/Learning-Models/blob/master/simple_activation_v1.py
#################### ham cheese forgetting DM model ###################

# this model turns on the subsymbolic processing for DM, which causes forgetting


##import ccm      
##log=ccm.log()   

##from ccm.lib.actr import *  

#####
# Python ACT-R requires an environment
# but in this case we will not be using anything in the environment
# so we 'pass' on putting things in there

from python_actr import *  

class MyEnvironment(Model):
    pass

#####
# create an act-r agent

class MyAgent(ACTR):
    focus=Buffer()

    DMbuffer=Buffer()                   
    DM=Memory(DMbuffer,latency=1.0,threshold=1)   # latency controls the relationship between activation and recall
                                                     # activation must be above threshold - can be set to none
            
    dm_n=DMNoise(DM,noise=0.0,baseNoise=0.0)         # turn on for DM subsymbolic processing
    dm_bl=DMBaseLevel(DM,decay=0.5,limit=None)       # turn on for DM subsymbolic processing


    def init():
        DM.add('customer:customer1 condiment:mustard')
        focus.set('rehearse')
                                        
    def request_chunk(focus='rehearse'):  
        print ("recalling the order")
        DM.request('customer:customer1 condiment:?condiment')            
        focus.set('recall') 

    def recall_chunk(focus='recall', DMbuffer='customer:customer1 condiment:?condiment'):  
        print ("Customer 1 wants.......")         
        print (condiment)                     # note - outside the quotes you do not need the ?       
        DM.add('customer:customer1 ?condiment')      # but inside you do   
        DMbuffer.clear()                    # each time you put something in memory (DM.add) it increases activation
        focus.set('rehearse')

    def forgot(focus='recall', DMbuffer=None, DM='error:True'):
                                           # DMbuffer=none means the buffer is empty
                                           # DM='error:True' means the search was unsucessful
        print ("I recall they wanted.......")
        print ("I forgot")
        focus.set('stop')



tim=MyAgent()                              # name the agent
subway=MyEnvironment()                     # name the environment
subway.agent=tim                           # put the agent in the environment
log_everything(subway)                 # print out what happens in the environment
#log=ccm.log(html=True)
subway.run(2)                               # run the environment FOR 2 SECONDS
#ccm.finished()                             # stop the environment

   0.400 agent.production_match_delay 0
   0.400 agent.production_threshold None
   0.400 agent.production_time 0.05
   0.400 agent.production_time_sd None
   0.400 agent.DM.error False
   0.400 agent.DM.busy False
   0.400 agent.DM.latency 1.0
   0.400 agent.DM.threshold 1
   0.400 agent.DM.maximum_time 10.0
   0.400 agent.DM.record_all_chunks False
   0.400 agent.DMbuffer.chunk None
   0.400 agent.focus.chunk None
   0.400 agent.focus.chunk rehearse
   0.400 agent.production request_chunk
   0.050 agent.production None
recalling the order
   0.050 agent.DM.busy True
   0.050 agent.focus.chunk recall
   0.274 agent.DMbuffer.chunk condiment:mustard customer:customer1
   0.274 agent.DM.busy False
   0.274 agent.production recall_chunk
   0.324 agent.production None
Customer 1 wants.......
mustard
   0.324 agent.DMbuffer.chunk None
   0.324 agent.focus.chunk rehearse
   0.324 agent.production request_chunk
   0.374 agent.production None
recalling the order
   0.374 agent.DM.busy True
   

In [ ]:
#https://github.com/rlwest/Learning-Models/blob/master/complexfanRLW1.ipynb
#import sys
#sys.path.append('/Users/robertwest/CCMSuite')
#sys.path.append('C:\Users\rlwes_000\Documents\GitHub\ccmsuite')

#import ccm      
#log=ccm.log()   

from python_actr import *  

class MyEnvironment(Model):
    pass

class MyAgent(ACTR):
    focus=Buffer()
    Imagebuffer=Buffer
    Visionbuffer=Buffer()
    DMbuffer=Buffer()
    DM=Memory(DMbuffer)

    
    def init():
        DM.add('isa:objectcontainer objectx:apple container:bowl')
        DM.add('isa:objectcontainer objectx:apple container:bucket')
        DM.add('isa:objectcontainer objectx:apple container:bin')
        
        DM.add('isa:containerlocation container:bowl location:house')
        DM.add('isa:containerlocation container:bucket location:store')
        DM.add('isa:containerlocation container:bin location:park')
        
        focus.set('status:start')
        Visionbuffer.set('isa:objectlocation objectx:apple location:house')


    #start trial
    def start(focus='status:start', Visionbuffer='isa:objectlocation objectx:?objectx'):
        print ("recalling based on objects")
        DM.request('isa:objectcontainer objectx:?objectx container:?')
        focus.set('status:get_container') 


    #recall a container for the target object
    def container(focus='status:get_container', 
    DMbuffer='isa:objectcontainer objectx:?objectx container:?container'):  
      #print objectx
      #print "is in ......."         
      print (container)
      DM.request('isa:containerlocation container:?container location:?')
      focus.set('status:get_location')
        

    #recall a location for the container the object is in
    def location(focus='status:get_location', 
    DMbuffer='isa:containerlocation container:?container location:?location'):  
      print (container)
      print ("is in .......")         
      print (location)
      focus.set('status:check_location')


    #success
    def yes_location(focus='status:check_location', 
    DMbuffer='container:?container location:?location', Visionbuffer='location:?location'):  
      print ('YES')
      focus.set('status:stop')


    #failure
    def no_location(focus='status:check_location', 
    DMbuffer='container:?container location:?location', Visionbuffer='location:!?location'):  
      print ('NO')
      focus.set('status:stop')



tim=MyAgent()                              # name the agent
subway=MyEnvironment()                     # name the environment
subway.agent=tim                           # put the agent in the environment
log_everything(subway)                 # print out what happens in the environment

subway.run()                               # run the environment
#ccm.finished()                             # stop the environment

   2.000 agent.production_match_delay 0
   2.000 agent.production_threshold None
   2.000 agent.production_time 0.05
   2.000 agent.production_time_sd None
   2.000 agent.DM.error False
   2.000 agent.DM.busy False
   2.000 agent.DM.latency 0.05
   2.000 agent.DM.threshold 0
   2.000 agent.DM.maximum_time 10.0
   2.000 agent.DM.record_all_chunks False
   2.000 agent.DMbuffer.chunk None
   2.000 agent.Imagebuffer.chunk None
   2.000 agent.Visionbuffer.chunk None
   2.000 agent.focus.chunk None
   2.000 agent.focus.chunk status:start
   2.000 agent.Visionbuffer.chunk isa:objectlocation location:house objectx:apple
   2.000 agent.production start
   0.050 agent.production None
recalling based on objects
   0.050 agent.DM.busy True
   0.050 agent.focus.chunk status:get_container
   0.100 agent.DMbuffer.chunk container:bin isa:objectcontainer objectx:apple
   0.100 agent.DM.busy False
   0.100 agent.production container
   0.150 agent.production None
bin
   0.150 agent.DM.busy True
   0.150

In [ ]:
#https://github.com/rlwest/Learning-Models/blob/master/simple_refreaction_v1.py
#################### ham cheese forgetting DM model ###################

##import sys
##sys.path.append('/Users/robertwest/CCMSuite')

##import ccm      
##log=ccm.log()  
##from ccm.lib.actr import *

#####
# Python ACT-R requires an environment
# but in this case we will not be using anything in the environment
# so we 'pass' on putting things in there

from python_actr import *  

class MyEnvironment(Model):
    pass

#####
# create an act-r agent

class MyAgent(ACTR):
    focus=Buffer()
    DMbuffer=Buffer()                   
    DM=Memory(DMbuffer,latency=0.05,threshold=-25,maximum_time=20,finst_size=10,finst_time=30.0)     
                                                    # turn down threshold
                                                    # maximum time - how long it will wait for a memory retrieval
                                                    # finst_size - how many chunks can be kept track of
                                                    # finst_time - how long a chunk can be kept track of
            
    dm_n=DMNoise(DM,noise=0.0,baseNoise=0.0)        
    dm_bl=DMBaseLevel(DM,decay=0.5,limit=None)   

    dm_spread=DMSpreading(DM,focus)                  
    dm_spread.strength=2                             
    dm_spread.weight[focus]=.5                       
                                                    
    partial=Partial(DM,strength=1.0,limit=-1.0)     
    partial.similarity('customer1','customer2',-0.1)   
    partial.similarity('customer1','customer3',-0.9)  

                    # note that this model uses slot names - slotname:slotcontent
    def init(): 
        DM.add('isa:order customer:customer1 type:ham_cheese condiment:mustard')         # customer1's order
        DM.add('isa:order customer:customer2 type:ham_cheese condiment:ketchup')         # customer2's order
        DM.add('isa:order customer:customer3 type:ham_cheese condiment:mayonnaise')      # customer3's order
        DM.add('isa:order customer:customer4 type:ham_cheese condiment:hot_sauce')       # customer4's order
        focus.set('isa:ingrediant type:bread')
        
    def bread_bottom(focus='isa:ingrediant type:bread'):   
        print ("I have a piece of bread")
        focus.set('isa:ingrediant type:cheese')
        
    def cheese(focus='isa:ingrediant type:cheese'):        
        print ("I have put cheese on the bread")  
        focus.set('isa:ingrediant type:ham')
        
    def ham(focus='isa:ingrediant type:ham'):
        print ("I have put  ham on the cheese")
        focus.set('isa:order customer:customer1 type:ham_cheese condiment:unknown')         
                                        
    def condiment(focus='isa:order customer:customer1 type:ham_cheese condiment:unknown'):
        print ("recalling the order")     
        DM.request('isa:order type:ham_cheese',require_new=True) # retrieve something that has not recently been retrieved           
        focus.set('retrieve_condiment')
        
    def order(focus='retrieve_condiment', DMbuffer='isa:order type:ham_cheese condiment:?condiment_order'): 
        print ("I recall they wanted.......")         
        print (condiment_order)            
        print ("i have put the condiment on the sandwich")
        focus.set('isa:ingrediant type:bread_top')
        
    def bread_top(focus='isa:ingrediant type:bread_top'):
        print ("I have put bread on the ham")
        print ("I have made a ham and cheese sandwich")
        focus.set('isa:ingrediant type:bread')
        DMbuffer.clear()                        # clear the buffer for the next cycle


tim=MyAgent()                              # name the agent
subway=MyEnvironment()                     # name the environment
subway.agent=tim                           # put the agent in the environment
log_everything(subway)                 # print out what happens in the environment
subway.run(3)                              # run the environment
#ccm.finished()                             # stop the environment

   3.150 agent.production_match_delay 0
   3.150 agent.production_threshold None
   3.150 agent.production_time 0.05
   3.150 agent.production_time_sd None
   3.150 agent.DM.error False
   3.150 agent.DM.busy False
   3.150 agent.DM.latency 0.05
   3.150 agent.DM.threshold -25
   3.150 agent.DM.maximum_time 20
   3.150 agent.DM.record_all_chunks False
   3.150 agent.DMbuffer.chunk None
   3.150 agent.focus.chunk None
   3.150 agent.focus.chunk isa:ingrediant type:bread
   3.150 agent.production bread_bottom
   0.050 agent.production None
I have a piece of bread
   0.050 agent.focus.chunk isa:ingrediant type:cheese
   0.050 agent.production cheese
   0.100 agent.production None
I have put cheese on the bread
   0.100 agent.focus.chunk isa:ingrediant type:ham
   0.100 agent.production ham
   0.150 agent.production None
I have put  ham on the cheese
   0.150 agent.focus.chunk condiment:unknown customer:customer1 isa:order type:ham_cheese
   0.150 agent.production condiment
   0.200 agent.

In [ ]:
#https://github.com/rlwest/Learning-Models/blob/master/ComplexFan-Kam.ipynb
#Complex fan retrieval
# Kam Kwok, 2014
##import ccm
##from ccm.lib.actr import *
#log=log(html=True)

from python_actr import *  

class CFanModel(ACTR):
    goal=Buffer()
    retrieval=Buffer()
    img=Buffer()
##    memory=Memory(retrieval,latency=0.63)
    memory=Memory(retrieval,latency=0.05,threshold=-25,maximum_time=20,finst_size=10,finst_time=0.0)
    spread=DMSpreading(memory,goal)
    spread.strength=1
    spread.weight[goal]=0.33
###Turning on spreading action and inhibition
##  memory=Memory(retrieval,latency=0.05,threshold=-25,maximum_time=20,finst_size=10,finst_time=0.0) # this line is copied from simple inhibition example      
##  dm_n=DMNoise(memory,noise=0.0,baseNoise=0.0)    # turn on for DM subsymbolic processing
##  dm_bl=DMBaseLevel(memory,decay=0.5,limit=None)  # turn on for DM subsymbolic processing; decay is default to 0.5 based on experiments
##  spread=DMSpreading(memory,goal)                  # turn on spreading activation for DM from goal buffer
##  spread.strength=1                             # set strength of activation for buffers
##  spread.weight[goal]=.33                       # set weight to adjust for how many slots in the buffer
##                                                 # usually this is strength divided by number of slots
    def init():
        memory.add('ring in bucket')
        memory.add('ring in box')
        memory.add('box in pool')

# probe is ?obj in ?location
    def start_obj(goal='test ?obj ?location'):
        memory.request('?obj in ?') #get contianer in retrieval
        goal.set('getcontainer ?obj ?location')
    
    def getcontainer(goal='getcontainer ?obj ?location',
                   retrieval='?obj in ?container'):
        retrieval.clear()
        memory.request('?container in ?')
        goal.set('respond ?obj ?location')

    def respond_yes(goal='respond ?obj ?location',
                  retrieval='?container in ?location'):
        print ('Yes')
        retrieval.clear()
        goal.clear()

    def respond_retry (goal='respond ?obj ?location',
                      retrieval='?container in !?location'):
        img.set('?container')
        print ("re-try", container)
        memory.request('?container in ?')
        goal.set('respond ?obj ?location')
##        retrieval.clear()
##        goal.set('test ?obj ?location')
  
    def respond_no(goal='respond ?obj ?location',
                      img='?container',
                      retrieval='?container in !?location'):
        print ('No')
        retrieval.clear()
        goal.clear()

    def mem_error(goal='respond ?obj ?location', memory='error:True'):
        print ("Cannot recall any fact...")
        goal.set('test ?obj ?location')


model=CFanModel()
log_everything(model)
##for item in ['test ring pool','test hippie park']:
##    model.goal.set(item)
##    model.run()
##model.goal.set('test ring bank')
##model.run()
model.goal.set('test ring pool')
model.run()
model.goal.set('test ring bank')
model.run()

   3.000 production_match_delay 0
   3.000 production_threshold None
   3.000 production_time 0.05
   3.000 production_time_sd None
   3.000 goal.chunk None
   3.000 img.chunk None
   3.000 memory.error False
   3.000 memory.busy False
   3.000 memory.latency 0.05
   3.000 memory.threshold -25
   3.000 memory.maximum_time 20
   3.000 memory.record_all_chunks False
   3.000 retrieval.chunk None
   3.000 goal.chunk test ring pool
   3.000 production start_obj
   0.050 production None
   0.050 memory.busy True
   0.050 goal.chunk getcontainer ring pool
   0.102 retrieval.chunk ring in box
   0.102 memory.busy False
   0.102 production getcontainer
   0.152 production None
   0.152 retrieval.chunk None
   0.152 memory.busy True
   0.152 goal.chunk respond ring pool
   0.197 retrieval.chunk box in pool
   0.197 memory.busy False
   0.197 production respond_yes
   0.247 production None
Yes
   0.247 retrieval.chunk None
   0.247 goal.chunk None
   0.247 goal.chunk test ring bank
   0.247 prod

In [ ]:
#https://github.com/rlwest/Learning-Models/blob/master/animals.py
from python_actr import *  


class MyEnvironment(Model):
    pass


class MyAgent(ACTR):
    
    focus=Buffer()
    DMbuffer=Buffer()

    DM=Memory(DMbuffer,latency=0.05,threshold=-0.6,maximum_time=3,finst_size=3,finst_time=3)
    #DM=Memory(DMbuffer)
    dm_bl=DMBaseLevel(DM,decay=0.5,limit=None)
    #DM_inhibit=DMInhibition(DM, decayScale=1.0, timeScale=5.0)
    
    focus.set('goal:recall')

    DM.add('animal:tiger')
    DM.add('animal:lion')
    #DM.add('animal:shark')
    #DM.add('animal:mouse')
    #DM.add('animal:duck')
    


    def request(focus='goal:recall'):
        print("requesting")
        DM.request('animal:?x')            
        focus.set('goal:retrieve')     

    def retrieve(DMbuffer='animal:?animal'): 
        print("+++++++++++++++++++++++++++++++++++++++++++")                       
        print (animal)
        DMbuffer.set('nil')
        focus.set('goal:recall')

   

tim=MyAgent()                              # name the agent
subway=MyEnvironment()                     # name the environment
subway.agent=tim                           # put the agent in the environment
log_everything(subway)
subway.run()                               # run the environment

  20.850 agent.production_match_delay 0
  20.850 agent.production_threshold None
  20.850 agent.production_time 0.05
  20.850 agent.production_time_sd None
  20.850 agent.DM.error False
  20.850 agent.DM.busy False
  20.850 agent.DM.latency 0.05
  20.850 agent.DM.threshold -0.6
  20.850 agent.DM.maximum_time 3
  20.850 agent.DM.record_all_chunks False
  20.850 agent.DMbuffer.chunk None
  20.850 agent.production request
   0.050 agent.production None
requesting
   0.050 agent.DM.busy True
   0.050 agent.focus.chunk goal:retrieve
   0.061 agent.DMbuffer.chunk animal:tiger
   0.061 agent.DM.busy False
   0.061 agent.production retrieve
   0.111 agent.production None
+++++++++++++++++++++++++++++++++++++++++++
tiger
   0.111 agent.DMbuffer.chunk nil
   0.111 agent.focus.chunk goal:recall
   0.111 agent.production request
   0.161 agent.production None
requesting
   0.161 agent.DM.busy True
   0.161 agent.focus.chunk goal:retrieve
   0.181 agent.DMbuffer.chunk animal:tiger
   0.181 agent.DM

In [ ]:
#https://github.com/rlwest/Learning-Models/blob/master/complexFan_productionsv2.py
#Complex fan retrieval productions, v.2
# Kam Kwok, 2014

##import sys
##sys.path.append('/Users/robertwest/CCMSuite')

##import ccm
##from ccm.lib.actr import *
##log=ccm.log(html=True)

from python_actr import *

class CFanModel(ACTR):
    goal=Buffer()
    retrieval=Buffer()
    img=Buffer()
##    memory=Memory(retrieval,latency=0.63)
    memory=Memory(retrieval,latency=0.05,threshold=-25,maximum_time=20,finst_size=10,finst_time=0.0)
    spread=DMSpreading(memory,goal)
    spread.strength=1
    spread.weight[goal]=0.33
#***
# with slots 
    def init():
        memory.add('objs:ring cont:bucket')
        memory.add('objs:ring cont:box')
        memory.add('cont:box loc:pool')

### slotted *** *** by obj
    def start_obj(goal='cmd:test objs:?obj loc:?location'):
        memory.request('objs:?obj cont:?') #get contianer in retrieval
        goal.set('cmd:usecontainer objs:?obj loc:?location')
    
    def useContGetlocation(goal='cmd:usecontainer objs:?obj loc:?location',
                   retrieval='objs:?obj cont:?container', memory='error:False'):
        retrieval.clear()
        memory.request('cont:?container loc:?')
        goal.set('cmd:respond objs:?obj loc:?location')

    def respond_yes(goal='cmd:respond objs:?obj loc:?location',
                  retrieval='cont:?container loc:?location', memory='error:False'):
        print ('Yes')
        retrieval.clear()
        goal.clear()
    #wrong location
    def respond_wloc_retryC (goal='cmd:respond objs:?obj loc:?location',
                      retrieval='cont:?container loc:!?location'):
        img.set('cont:?container')
        print ("Wrong location, re-try with container", container)
        memory.request('cont:?container loc:?')
        goal.set('cmd:respond objs:?obj loc:?location')

    #Retry - no location retry with object   
    def respond_noloc_retryO (goal='cmd:respond objs:?obj loc:?location',
                      retrieval='cont:?container'):
        img.set('cont:?container')
        print ("No location, re-try for another container")
        retrieval.clear()
        goal.set('cmd:test objs:?obj loc:?location')
    #retrieved the same container and retrieved no location
    # Retry O to get another C
    def respond_retryC_RetryOL(goal='cmd:respond objs:?obj loc:?location',
                      img='cont:?container',
                      retrieval='cont:?container'):
        print ('Get another container')
        retrieval.clear()
        goal.set('cmd:test objs:?obj loc:?location')
    # Say No:after retry, got a wrong location with the repeated container    
    def respond_No_wloc_img(goal='cmd:respond objs:?obj loc:?location',
                      img='cont:?container',
                      retrieval='cont:?container loc:!?location'):
        print ('No')
        retrieval.clear()
        goal.clear()

    def mem_error(goal='cmd:respond objs:?obj loc:?location',
                  img='cont:?container',
                  memory='error:True'):
        print ("Cannot recall any fact...so no!")
        retrieval.clear()
        goal.clear()

#### slotted *** *** by location
#********************
    def start_loc(goal='cmd:testl objs:?obj loc:?location'):
        memory.request('cont:? loc:?location') #get contianer in retrieval
        goal.set('cmd:getobj objs:?obj loc:?location')
    
    def useContGetObj(goal='cmd:getobj objs:?obj loc:?location',
                   retrieval='cont:?container loc:?location', memory='busy:False'):
        retrieval.clear()
        memory.request('objs:? cont:?container')
        goal.set('cmd:respondl objs:?obj loc:?location')

    def respond_yes_loc(goal='cmd:respondl objs:?obj loc:?location',
                  retrieval='objs:?obj cont:?container', memory='error:False'):
        print ('Yes by location')
        retrieval.clear()
        goal.clear()
    #wrong object
    def respond_wobj_retryC (goal='cmd:respondl objs:?obj loc:?location',
                      retrieval='objs:!?obj cont:?container'):
        img.set('cont:?container')
        print ("Wrong object, re-try with container", container)
        memory.request('objs:? cont:?container')
        goal.set('cmd:respondl objs:?obj loc:?location')

    #Retry - no object retry with object   
    def respond_noobj_retryO (goal='cmd:respondl objs:?obj loc:?location',
                      retrieval='cont:?container'):
        img.set('cont:?container')
        print ("No object, re-try for another container")
        retrieval.clear()
        goal.set('cmd:testl objs:?obj loc:?location')
    #retrieved the same container and retrieved no object
    # Retry O to get another C
    def respond_retryC_RetryOL(goal='cmd:respondl objs:?obj loc:?location',
                      img='cont:?container',
                      retrieval='cont:?container'):
        print ('Get another container')
        retrieval.clear()
        goal.set('cmd:testl objs:?obj loc:?location')
    # Say No:after retry, got a wrong object with the repeated container    
    def respond_No_wobj_img(goal='cmd:respondl objs:?obj loc:?location',
                      img='cont:?container',
                      retrieval='objs:!?obj cont:?container'):
        print ('No')
        retrieval.clear()
        goal.clear()

    def mem_error_l(goal='cmd:respondl objs:?obj loc:?location',
                  img='cont:?container',
                  memory='error:True'):
        print ("Cannot recall any fact...so no!")
        retrieval.clear()
        goal.clear()

        
#*** *** by obj
        
### no slot ******
##    def init():
##        memory.add('ring in bucket')
##        memory.add('ring in box')
##        memory.add('box in pool')
##
### probe is ?obj in ?location
##    def start_obj(goal='test ?obj ?location'):
##        memory.request('?obj in ?') #get contianer in retrieval
##        goal.set('getcontainer ?obj ?location')
##    
##    def getcontainer(goal='getcontainer ?obj ?location',
##                   retrieval='?obj in ?container', memory='error:False'):
##        retrieval.clear()
##        memory.request('?container in ?')
##        goal.set('respond ?obj ?location')
##
##    def respond_yes(goal='respond ?obj ?location',
##                  retrieval='?container in ?location', memory='error:False'):
##        print 'Yes'
##        retrieval.clear()
##        goal.clear()
##
##    def respond_retry (goal='respond ?obj ?location',
##                      retrieval='?container in !?location',memory='error:False'):
##        img.set('?container')
##        print "re-try with container", container
##        memory.request('?container in ?')
##        goal.set('respond ?obj ?location')
####        retrieval.clear()
####        goal.set('test ?obj ?location')
##  
##    def respond_no(goal='respond ?obj ?location',
##                      img='?container',
##                      retrieval='?container in !?location'):
####    def respond_no(goal='respond ?obj ?location',
####                      retrieval='?container in !?location'):
##        print 'No'
##        retrieval.clear()
##        goal.clear()
##
##    def mem_error(goal='respond ?obj ?location', memory='error:True'):
##        print "Cannot recall any fact...so no!"
####        goal.set('test ?obj ?location')
##        retrieval.clear()
##        goal.clear()

##        # by location
##
##    def start_loc(goal='test byloc ?obj ?location'):
##        memory.request('? in ?location') #c
##        goal.set('getcontainerl ?obj ?location')#c
##    
##    def getcontainerl(goal='getcontainerl ?obj ?location',
##                   retrieval='?container in ?location', memory='busy:False'):
##        retrieval.clear()
##        memory.request('? in ?container')
##        goal.set('respondl ?obj ?location')
##
##    def respondl_yes(goal='respondl ?obj ?location',
##                  retrieval='?obj in ?container'):
##        print 'Yes'
##        retrieval.clear()
##        goal.clear()
##
##    def respondl_retry (goal='respondl ?obj ?location',
##                      retrieval='!?obj in ?container',memory='busy:False'):
##        img.set('?container')
##        print "re-try location", container
##        memory.request('? in ?container')
##        goal.set('respondl ?obj ?location')
####        retrieval.clear()
####        goal.set('test ?obj ?location')
##  
##    def respondl_no(goal='respondl ?obj ?location',
##                      img='?container',
##                      retrieval='!?obj in ?container'):
##        print 'No'
##        retrieval.clear()
##        goal.clear()
##
##    def mem_error(goal='? ? ?', memory='error:True'):
##        print "Cannot recall any fact, I'll say no."
##        retrieval.clear()
##        goal.clear()


model=CFanModel()
log_everything(model)
##for item in ['test ring pool','test hippie park']:
##    model.goal.set(item)
##    model.run()
##model.goal.set('test ring bank')
##model.run()
##model.goal.set('test ring pool')
##model.run()
model.goal.set('cmd:test objs:ring loc:pool')
model.run()
model.goal.set('cmd:test objs:ring loc:bank')
model.run()
##model.goal.set('cmd:testl objs:ring loc:pool')
##model.run()
##model.goal.set('cmd:testl objs:ring loc:bank')
##model.run()

   6.154 production_match_delay 0
   6.154 production_threshold None
   6.154 production_time 0.05
   6.154 production_time_sd None
   6.154 goal.chunk None
   6.154 img.chunk None
   6.154 memory.error False
   6.154 memory.busy False
   6.154 memory.latency 0.05
   6.154 memory.threshold -25
   6.154 memory.maximum_time 20
   6.154 memory.record_all_chunks False
   6.154 retrieval.chunk None
   6.154 goal.chunk cmd:test loc:pool objs:ring
   6.154 production start_obj
   0.050 production None
   0.050 memory.busy True
   0.050 goal.chunk cmd:usecontainer loc:pool objs:ring
   0.102 retrieval.chunk cont:box objs:ring
   0.102 memory.busy False
   0.102 production useContGetlocation
   0.152 production None
   0.152 retrieval.chunk None
   0.152 memory.busy True
   0.152 goal.chunk cmd:respond loc:pool objs:ring
   0.197 retrieval.chunk cont:box loc:pool
   0.197 memory.busy False
   0.197 production respond_yes
   0.247 production None
Yes
   0.247 retrieval.chunk None
   0.247 goal.c

In [ ]:
#

In [ ]:
class RepeatedBinaryChoiceModel(ACTR):
    goal=Buffer()
    
    pmnoise=PMNoise(noise=0.3)
    pm=PMNew(alpha=0.2)
    
    def pressA(goal='action:choose'):
        self.reward(0.3)
        
    def pressB(goal='action:choose'):
        self.reward(0.1)
        
        
# run the model        
model=RepeatedBinaryChoiceModel()
log_everything(model)
model.goal.set('action:choose')
model.reward
model.run(limit=1.5)


   0.598 production_match_delay 0
   0.598 production_threshold None
   0.598 production_time 0.05
   0.598 production_time_sd None
   0.598 goal.chunk None
   0.598 pm.alpha 0.2
   0.598 pm.clearFlag False
   0.598 pmnoise.noise 0.3
   0.598 pmnoise.baseNoise 0.0
   0.598 goal.chunk action:choose
   0.598 production pressA
   0.050 production None
   0.050 pm.clearFlag True
   0.050 pm.clearFlag False
   0.050 production pressA
   0.100 production None
   0.100 pm.clearFlag True
   0.100 pm.clearFlag False
   0.100 production pressB
   0.150 production None
   0.150 pm.clearFlag True
   0.150 pm.clearFlag False
   0.150 production pressA
   0.200 production None
   0.200 pm.clearFlag True
   0.200 pm.clearFlag False
   0.200 production pressB
   0.250 production None
   0.250 pm.clearFlag True
   0.250 pm.clearFlag False
   0.250 production pressA
   0.300 production None
   0.300 pm.clearFlag True
   0.300 pm.clearFlag False
   0.300 production pressA
   0.350 production None
   0.35

In [ ]:
print(model.reward.func)

<function ACTR.reward at 0x7f15b62c6710>


In [ ]:
print(model.reward)

In [ ]:
#%%capture cap
mymap="""
################
#              #
#              #
#              #
######### ######
#     #        #
#     #        #
#              #
################
"""
class MyEnvironment(Model):
    def color(self):
        if self.wall: return 'black'
        else: return 'white'
    def load(self,char):      
        if char=='#': self.wall=True

class MyAgentABC(ACTR):
  goal = Buffer()
  DMBuffer = Buffer()
  DM = Memory(DMBuffer)
  def init():
    DM.add('Say A B')
    DM.add('Say B C')
    DM.add('Say C D')
    DM.add('Say D E')
    DM.add('Say E F')
    DM.add('Say F G')
    goal.set('sayFrom C F starting')

  def start(goal='sayFrom ?start ?end starting'):
    DM.request('Say ?start ?next')
    goal.set('sayFrom ?start ?end saying')
  def increment(goal='sayFrom ?x !?x saying',DMBuffer='Say ?x ?next'):
    print (" The current letter is:", x, "\n")
    DM.request('Say ?next ?nextNext')
    goal.modify(_1=next)
  def stop(goal='sayFrom ?x ?x saying'):
    print (" The final letter is:", x, "\n")
    goal.set('sayFrom ?x ?x stop')

tim=MyAgentABC()                              # name the agent
subway=MyEnvironment()                     # name the environment
subway.agent=tim                           # put the agent in the environment
log_everything(subway)                 # print out what happens in the environment
subway.run()                               # run the environment
#ccm.finished()                             # stop the environment

   1.500 agent.production_match_delay 0
   1.500 agent.production_threshold None
   1.500 agent.production_time 0.05
   1.500 agent.production_time_sd None
   1.500 agent.DM.error False
   1.500 agent.DM.busy False
   1.500 agent.DM.latency 0.05
   1.500 agent.DM.threshold 0
   1.500 agent.DM.maximum_time 10.0
   1.500 agent.DM.record_all_chunks False
   1.500 agent.DMBuffer.chunk None
   1.500 agent.goal.chunk None
   1.500 agent.goal.chunk sayFrom C F starting
   1.500 agent.production start
   0.050 agent.production None
   0.050 agent.DM.busy True
   0.050 agent.goal.chunk sayFrom C F saying
   0.100 agent.DMBuffer.chunk Say C D
   0.100 agent.DM.busy False
   0.100 agent.production increment
   0.150 agent.production None
 The current letter is: C 

   0.150 agent.DM.busy True
   0.150 agent.goal.chunk sayFrom D F saying
   0.200 agent.DMBuffer.chunk Say D E
   0.200 agent.DM.busy False
   0.200 agent.production increment
   0.250 agent.production None
 The current letter is: D 



In [ ]:
%%capture cap

from python_actr import *  

class MyEnvironment(Model):
    pass
class MyAgent(ACTR):
     
    focus=Buffer()
    focus.set('sandwich_step:bread_bottom')

    def bread_bottom(focus='sandwich_step:bread_bottom'):# if focus buffer has this chunk then....
        print("I have a piece of bread")                 # print the action
        focus.set('sandwich_step:cheese')                # change chunk in focus buffer

    def cheese(focus='sandwich_step:cheese'):            # the rest of the productions are the same
        print("I have put cheese on the bread")          # but carry out different actions
        focus.set('sandwich_step:ham')

    def ham(focus='sandwich_step:ham'):
        print("I have put  ham on the cheese")
        focus.set('sandwich_step:bread_top')

    def bread_top(focus='sandwich_step:bread_top'):
        print("I have put bread on the ham")
        print("I have made a ham and cheese sandwich")
        focus.set('sandwich_step:stop')   

tim=MyAgent()                              # name the agent
subway=MyEnvironment()                     # name the environment
subway.agent=tim                           # put the agent in the environment
log_everything(subway)
subway.run() 

In [ ]:
print (cap)

   3.350 agent.production_match_delay 0
   3.350 agent.production_threshold None
   3.350 agent.production_time 0.05
   3.350 agent.production_time_sd None
   3.350 agent.production bread_bottom
   0.050 agent.production None
I have a piece of bread
   0.050 agent.focus.chunk sandwich_step:cheese
   0.050 agent.production cheese
   0.100 agent.production None
I have put cheese on the bread
   0.100 agent.focus.chunk sandwich_step:ham
   0.100 agent.production ham
   0.150 agent.production None
I have put  ham on the cheese
   0.150 agent.focus.chunk sandwich_step:bread_top
   0.150 agent.production bread_top
   0.200 agent.production None
I have put bread on the ham
I have made a ham and cheese sandwich
   0.200 agent.focus.chunk sandwich_step:stop



In [ ]:
class Subway(Model):        
    coffee_machine = Model(isa='coffee_machine', button='up')
    cup = Model(isa='cup', state='empty', location='coffeemachine')

class ActionModule(Model):     

    def human_action(self, action, env_object, slot_name, slot_value):
        if action=='push':
            yield 1
        elif action=='grab':
            yield 2
        x = self.parent.parent[env_object]
        setattr(x, slot_name, slot_value)
        print('barista actions')
        print('   object=',env_object)
        print('   slot=',slot_name)
        print('   value=',slot_value)

    def coffeemachine_action(self, action, env_object, slot_name, slot_value):
        x = self.parent.parent[env_object]
        if action=='pour':
            setattr(x, slot_name, 'in_progress')
            yield 4
        setattr(x, slot_name, slot_value)
        print('coffee machine actions')
        print('   object=',env_object)
        print('   slot=',slot_name)
        print('   value=',slot_value)
        #print(action)
class Human(ACTR):    
    focus=Buffer()
    focus.set('state:start')
    action=ActionModule()

    def START(focus='state:start',cup='state:empty'):
        print('pushing coffee machine button')
        action.human_action('push','coffee_machine', 'button', 'down')
        focus.set('state:wait')

    def Grab_coffee(focus='state:wait',cup='state:full'):
        print('grabbing the coffee')
        action.human_action('grab','cup', 'location', 'hand')
        focus.set('state:stop')


class Coffee_Machine(ACTR):
    production_time=0.0
    focus=Buffer()
    focus.set('state:off')
    action=ActionModule()

    def On(focus='state:off',coffee_machine='button:down'):
        print('pouring coffee')
        action.coffeemachine_action('pour','cup', 'state', 'full')
        focus.set('state:on')

    def Off(focus='state:on',cup='state:full'):
        print('finished pouring coffee')
        action.coffeemachine_action('reset','coffee_machine', 'button', 'up')
        focus.set('state:off')


tim=Human()
machine1=Coffee_Machine()
env=Subway()
env.agent=tim
env.agent=machine1 

log_everything(env)

env.run()

   0.200 agent.production_match_delay 0
   0.200 agent.production_threshold None
   0.200 agent.production_time 0.0
   0.200 agent.production_time_sd None
   0.200 coffee_machine.isa coffee_machine
   0.200 coffee_machine.button up
   0.200 cup.isa cup
   0.200 cup.state empty
   0.200 cup.location coffeemachine
pushing coffee machine button
   1.050 coffee_machine.button down
barista actions
   object= coffee_machine
   slot= button
   value= down
   1.050 agent.production On
   1.050 agent.production None
pouring coffee
   1.050 agent.focus.chunk state:on
   1.050 cup.state in_progress
   5.050 cup.state full
coffee machine actions
   object= cup
   slot= state
   value= full
   5.050 agent.production Off
   5.050 agent.production None
finished pouring coffee
   5.050 agent.focus.chunk state:off
   5.050 coffee_machine.button up
coffee machine actions
   object= coffee_machine
   slot= button
   value= up
grabbing the coffee
   7.100 cup.location hand
barista actions
   object= cup
 

In [ ]:
from python_actr import *  

class Subway(Model):        
    coffee_machine = Model(isa='coffee_machine', button='up')
    cup = Model(isa='cup', state='empty', location='coffeemachine')

class ActionModule(Model):     

##    def human_action(self, action, env_object, slot_name, slot_value):
##        if action:='push':
##            print('PUSH',action)
##            yield 5
##        elif action:='grab':
##            print('GRAB',action)
##            yield 2
##        x = self.parent.parent[env_object]
##        setattr(x, slot_name, slot_value)
##        print('Human actions')
##        print('-object=',env_object)
##        print('-slot=',slot_name)
##        print('-value=',slot_value)

    def push(self, env_object, slot_name, slot_value):
        x = self.parent.parent[env_object]
        setattr(x, slot_name, 'in_progress')
        yield 2
        x = self.parent.parent[env_object]
        setattr(x, slot_name, slot_value)
        print('-object=',env_object)
        print('-slot=',slot_name)
        print('-value=',slot_value)
    def grab(self, env_object, slot_name, slot_value):
        x = self.parent.parent[env_object]
        setattr(x, slot_name, 'in_progress')
        yield 3
        x = self.parent.parent[env_object]
        setattr(x, slot_name, slot_value)
        print('-object=',env_object)
        print('-slot=',slot_name)
        print('-value=',slot_value)
    def pour(self, env_object, slot_name, slot_value):
        x = self.parent.parent[env_object]
        setattr(x, slot_name, 'in_progress')
        yield 5
        setattr(x, slot_name, slot_value)
        print('-object=',env_object)
        print('-slot=',slot_name)
        print('-value=',slot_value)
    def reset(self, env_object, slot_name, slot_value):
        x = self.parent.parent[env_object]
        # no yield
        setattr(x, slot_name, slot_value)
        print('-object=',env_object)
        print('-slot=',slot_name)
        print('-value=',slot_value)
        
class Human(ACTR):    
    focus=Buffer()
    focus.set('state:start')
    action=ActionModule()

    def START(focus='state:start',cup='state:empty'):
        print('HUMAN:pushing coffee machine button')
        action.push('coffee_machine', 'button', 'down')
        focus.set('state:wait')

    def Grab_coffee(focus='state:wait',cup='state:full'):
        print('HUMAN:grabbing the coffee')
        action.grab('cup', 'location', 'hand')
        focus.set('state:stop')


class Coffee_Machine(ACTR):
    production_time=0.0
    focus=Buffer()
    focus.set('state:off')
    action=ActionModule()

    def On(focus='state:off',coffee_machine='button:down'):
        print('COFFEE_MACHINE:pouring coffee')
        action.pour('cup', 'state', 'full')
        focus.set('state:on')

    def Off(focus='state:on',cup='state:full'):
        print('COFFEE_MACHINE:finished pouring coffee')
        action.reset('coffee_machine', 'button', 'up')
        focus.set('state:off')


tim=Human()
machine1=Coffee_Machine()
env=Subway()
env.agent=tim
env.agent=machine1 

log_everything(env)

env.run()

   7.100 agent.production_match_delay 0
   7.100 agent.production_threshold None
   7.100 agent.production_time 0.0
   7.100 agent.production_time_sd None
   7.100 coffee_machine.isa coffee_machine
   7.100 coffee_machine.button up
   7.100 cup.isa cup
   7.100 cup.state empty
   7.100 cup.location coffeemachine
HUMAN:pushing coffee machine button
   0.050 coffee_machine.button in_progress
   2.050 coffee_machine.button down
-object= coffee_machine
-slot= button
-value= down
   2.050 agent.production On
   2.050 agent.production None
COFFEE_MACHINE:pouring coffee
   2.050 agent.focus.chunk state:on
   2.050 cup.state in_progress
   7.050 cup.state full
-object= cup
-slot= state
-value= full
   7.050 agent.production Off
   7.050 agent.production None
COFFEE_MACHINE:finished pouring coffee
   7.050 coffee_machine.button up
-object= coffee_machine
-slot= button
-value= up
   7.050 agent.focus.chunk state:off
HUMAN:grabbing the coffee
   7.100 cup.location in_progress
  10.100 cup.locatio